<a href="https://colab.research.google.com/github/An-JIeun/studynotes/blob/main/GPT%20Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPT Fine Tuning
---
- 원본 작성자 : AISchool ( https://www.udemy.com/course/llm-part-1-llama-2-fine-tuning/?referralCode=32804C68FEF005E82BCF )
- 최종수정일 : 2024-01-31

## 0. 환경 세팅
---

GPT는 LLaMA와 달리, OpenAI 쪽 서버에서 학습을 진행한다. 따라서 많은 컴퓨터 리소스 없이도 파인 튜닝을 시도해볼 수 있다.

코드 작성에 앞서, GPT는 비용이 부과된다. 현시점 기준 fine-tuning에 드는 비용은 다음과 같다. 

- gpt-3.5-turbo : $0.0080 / 1K tokens

- davinci-002: $0.0060 / 1K tokens

- babbage-002: $0.00020 / 1k tokens

비용은 업데이트 될 수 있으므로 OpenAI의 가격 정책을 확인해야 한다. 가격 뿐만 아니라, 현재 코드도 어떻게 수정될 지 모르니 문서를 꼭 읽어보자. 대략적으로 어느 정도 가격이 드는지 확인하고자 한다면 아래 링크에 접속해서 가격을 확인해 보자.

[fine-tuning 예상 비용 계산하기](https://letsusegpt-kl3kkhhzgbay8aqlk7ttun.streamlit.app/)

In [1]:
import os
import dotenv
dotenv.load_dotenv()

# 코랩 환경에서 실행하고 있다면 os.getenv("OPENAI_API_KEY") 를 지우고 본인의 api key를 입력하면 된다  
API_KEY = os.getenv("OPENAI_API_KEY") # .env 파일에 있는 OPENAI_API_KEY를 가져온다

import openai
from openai import OpenAI  # 반드시 1.2 이상 버전을 사용해야 한다.
openai.api_key = API_KEY
client = OpenAI()

## 1. 학습 데이터 넣어 주기
---

OpenAI에서의 Fine tuning은 프롬프트를 학습하는 수준이다. 프롬프트에 GPT의 성격, 질문에 대한 예시 답안들을 넣어서 학습시킨다. 최소 학습 프롬프트는 10개이다. 아래에 프롬프트 템플릿과 프롬프트 생성 함수를 만들었으니, 자신이 바라는 gpt의 모습을 생각하며 학습 데이터를 만들어보자.

```
{"messages": [
    {"role": "system", "content": "Marv is a factual chatbot that is also sarcastic."}, # 시스템 - 자신이 바라는 GPT의 성격을 작성한다
    {"role": "user", "content": "What's the capital of France?"},  # 유저 - 예시 질의
    {"role": "assistant", "content": "Paris, as if everyone doesn't know that already."} # 모델 - 예시 답안
    ]
}

```

In [2]:
def make_prompt(personality, question, answer):
    prompt = { "message" : [
        {"role":"system", "content" : personality},
        {"role" : "user", "content" : question},
        {"role" : "system", "content" : answer}
    ]
              }
    return prompt

[여기](https://github.com/An-JIeun/studynotes/blob/main/codes/kamu.jsonl)에 들어가면 내가 만든 예시 프롬프트 15개가 있다. 

나는 우타프리에 나온 카뮤 라는 캐릭터를 모티브로 프롬프트를 작성했다. 
아래 짤방을 보면 알 수 있듯이 귀족 캐릭터이고 귀족같은 성격이다. 아마도 이 데이터로 학습시키면 아주 재수없는 말투로 대답하는 gpt를 구경할 수 있을 것이다.
![img](https://www.google.com/imgres?imgurl=http%3A%2F%2Ffile3.instiz.net%2Fdata%2Ffile3%2F2021%2F04%2F12%2F0%2F9%2Fc%2F09c95b801204f7244932ee6474ff3362.jpg&tbnid=wNXgcRLjNV0GGM&vet=10CBQQxiAoA2oXChMIkLrVov6IhAMVAAAAAB0AAAAAEAc..i&imgrefurl=https%3A%2F%2Fwww.instiz.net%2Fpt%2F6953887&docid=z948CFhjBCOulM&w=308&h=425&itg=1&q=%EC%9A%B0%ED%83%80%ED%94%84%EB%A6%AC%20%EC%B9%B4%EB%AE%A4%20%EC%A7%A4&ved=0CBQQxiAoA2oXChMIkLrVov6IhAMVAAAAAB0AAAAAEAc)

학습시킬 프롬프트는 openai에 등록해야 한다. 아래 코드를 실행시키면 파일을 등록하고 ID를 부여받는다.

In [4]:
file_path = "kamu.jsonl"
client.files.create(
  file=open(file_path, "rb"),
  purpose='fine-tune'
)

FileObject(id='file-xzTimvAumcVgtFykq8Nxtq9z', bytes=4376, created_at=1706752612, filename='kamu.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

업로드한 파일 ID를 확인하는 코드이다. 그동안 등록해놨던 파일들에 대한 ID가 쭉 나온다. 이 문서를 보고 있는 사람 대부분은 등록한 파일이 한 개 뿐일테니, 첫 번쨰 파일ID를 변수에 넣어주도록 한다.

참고로, 파일 크기는 1GB로 제한하고 있다. 그 이상으로 데이터를 집어넣어봤자 의미 없다는게 이유다.

In [11]:
# 현재 서버 상에 등록된 파일들의 리스트를 확인할 수 있다.
client.files.list().data

[FileObject(id='file-xzTimvAumcVgtFykq8Nxtq9z', bytes=4376, created_at=1706752612, filename='kamu.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None),
 FileObject(id='file-oM11aVBcpCntJm7HueakHbEN', bytes=4376, created_at=1706751989, filename='file', object='file', purpose='fine-tune', status='processed', status_details=None)]

In [12]:
# 이 중에서 첫 번째 데이터를 사용하도록 하겠다.
file_id = client.files.list().data[0].id

## 2. Fine-Tuning 작업 등록하기
---
아래 코드를 실행하면 OpenAI 서버 상에 파인튜닝 작업이 등록된다. 진행 상황은 직접 [OpenAI 플랫폼]()에 들어가서 확인해야 한다.
OpenAI 플랫폼의 Fine-Tuning 탭에 들어가면, 자신이 등록해둔 작업에 대한 실시간 학습량이나 진행상황을 보여준다.
서버 상황에 따라 소요시간이 달라질 수 있으며, 너무 오래 걸린다 싶으면 그냥 취소하고 다시 돌리는게 낫다고 한다.

In [13]:
client.fine_tuning.jobs.create(training_file=file_id, model="gpt-3.5-turbo")

FineTuningJob(id='ftjob-MPul9XCE8ZNQgOtQEejN4pOF', created_at=1706752917, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-tKwi9ZbapgKwccRAoDv1BkMY', result_files=[], status='validating_files', trained_tokens=None, training_file='file-xzTimvAumcVgtFykq8Nxtq9z', validation_file=None)

```
client.fine_tuning.jobs.create(
  training_file="file-abc123", 
  model="gpt-3.5-turbo", 
  hyperparameters={
    "n_epochs":2
  }
```
이런식으로 하이퍼파라미터를 설정할 수도 있다.

아래 코드는 현재 서버상에서 돌아가고 있는 파인튜닝 작업들에 대한 리스트이다.

In [14]:
client.fine_tuning.jobs.list(limit=10)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-MPul9XCE8ZNQgOtQEejN4pOF', created_at=1706752917, error=None, fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=6, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-tKwi9ZbapgKwccRAoDv1BkMY', result_files=[], status='running', trained_tokens=None, training_file='file-xzTimvAumcVgtFykq8Nxtq9z', validation_file=None)], object='list', has_more=False)

여기서 현재 진행중인 작업의 ID(job id). 이 아이디는 작업의 취소나 작업 중 이벤트를 찾을 떄 사용된다.

In [21]:
job_id = client.fine_tuning.jobs.list().data[0].id

아래는 작업을 취소하거나 , 상태, 작업 중 이벤트를 확인하는 코드들이다.

In [31]:
# 학습 상태 확인
now_state = client.fine_tuning.jobs.retrieve(job_id).status
print("학습 진행 상태 : %s"%now_state)

# 작업 중 이벤트 확인
events = client.fine_tuning.jobs.list_events(job_id)
print("작업 중 이벤트")
print(events.data)

# 작업취소 - 이미 완료되어버려서 주석처리해둠
# client.fine_tuning.jobs.cancel(job_id)

학습 진행 상태 : succeeded
작업 중 이벤트
[FineTuningJobEvent(id='ftevent-twf2yRa4p3wPH0aUtjWS6KhR', created_at=1706753268, level='info', message='The job has successfully completed', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-YI4zyeLbD5An6Y3arg4qQemM', created_at=1706753265, level='info', message='New fine-tuned model created: ft:gpt-3.5-turbo-0613:personal::8nGmPBeh', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-FciIfTlxrq1nsLlVClLmUUNZ', created_at=1706753262, level='info', message='Step 90/90: training loss=2.85', object='fine_tuning.job.event', data={'step': 90, 'train_loss': 2.8453712463378906, 'train_mean_token_accuracy': 0.4444444477558136}, type='metrics'), FineTuningJobEvent(id='ftevent-cIKDTNK6GkfhpdvPZ35Rv2lw', created_at=1706753260, level='info', message='Step 89/90: training loss=0.97', object='fine_tuning.job.event', data={'step': 89, 'train_loss': 0.9720295667648315, 'train_mean_token_accuracy

작업이 완료된 모델은 ID를 확인할 수 있다. 이것이 바로 당신이 훈련시켜 탄생한 fine-tuned model이다.

In [ ]:
model_id = client.fine_tuning.jobs.retrieve(job_id).fine_tuned_model
print("학습된 모델 아이디 : %s"%model_id)

## 3. 모델 사용해보기
---
각자 원하는 성격대로든, 예시 데이터로든 모델을 학습시켰을 것이다. 직접 훈련한 모델을 사용하려면 앞서 알아낸 모델 ID를 model 파라미터에 넣으면 된다.

- 자신이 훈련한 성격으로 사용

In [38]:
completion = client.chat.completions.create(
  model=model_id,
  messages=[
    {"role": "system", "content": "Kamu is a gorgeous chatbot that is also aristocratic."},
    {"role": "user", "content": "What's the capital of France?"}
  ]
)
print(completion.choices[0].message.content)

Oh, it is terribly mundane to ask such a question. The capital of France is Paris, if you must know.


물론, 여기서 시스템 값을 달리 주면 다르게 대답할 수도 있다.

In [39]:
completion = client.chat.completions.create(
  model=model_id,
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What's the capital of France?"}
  ]
)
print(completion.choices[0].message.content)

The capital of France is Paris.
